VISTO LEZ 10 anche se non spiega niente
spiega solo come usa i suoi rowGenerator e datasetGenerator
VISTO LEZ 12 (i commenti alle celle li ha fatti qua FINO A JOIN OVER TIME)
VISTO LEZ 13 (da join over time)

In [1]:
#remember to clone https://github.com/brcondor/Architectures_for_Big_Data into '/home/jovyan/work/'
import sys
sys.path.append("/home/jovyan/work/Architectures_for_Big_Data/")
import pyspark
sc = pyspark.SparkContext("local[3]")

In [2]:
from dataGenerator.rowGenerator import *
from dataGenerator.datasetGenerator import *

# genera dati a caso con i suoi generatori

dataset = datasetGenerator()
dataset.addGenerator(idGenerator(),prefix="line",min=1,max=5,keyName="lineId")
dataset.addGenerator(idGenerator(),prefix="item",min=1,max=15,keyName="itemId")
dataset.addGenerator(tickGenerator(),minTick=600,maxTick=1800,keyName="creationDate")
dataset.addGenerator(intGenerator(),keyName="qty")

logRdd = sc.parallelize( dataset.generateDataset(25000)).persist()
logRdd.first()

{'lineId': 'line_04',
 'itemId': 'item_010',
 'creationDate': datetime.datetime(2010, 1, 1, 0, 2, 51),
 'qty': 91}

In [3]:
# lets see the oldest entry
logRdd.map(lambda x: x.get("creationDate")).sortBy(lambda x: x).map(lambda x: str(x)).take(5)

['2010-01-01 00:29:14',
 '2010-01-01 00:40:41',
 '2010-01-01 00:46:42',
 '2010-01-01 00:48:52',
 '2010-01-01 01:04:44']

In [23]:
# lets check that all couples line-item are there
logRdd.map(lambda x: (x.get("lineId"),x.get("itemId"))).distinct().sortBy(lambda x: (x[0],x[1])).collect()

[('line_01', 'item_001'),
 ('line_01', 'item_002'),
 ('line_01', 'item_003'),
 ('line_01', 'item_004'),
 ('line_01', 'item_005'),
 ('line_01', 'item_006'),
 ('line_01', 'item_007'),
 ('line_01', 'item_008'),
 ('line_01', 'item_009'),
 ('line_01', 'item_010'),
 ('line_01', 'item_011'),
 ('line_01', 'item_012'),
 ('line_01', 'item_013'),
 ('line_01', 'item_014'),
 ('line_01', 'item_015'),
 ('line_02', 'item_001'),
 ('line_02', 'item_002'),
 ('line_02', 'item_003'),
 ('line_02', 'item_004'),
 ('line_02', 'item_005'),
 ('line_02', 'item_006'),
 ('line_02', 'item_007'),
 ('line_02', 'item_008'),
 ('line_02', 'item_009'),
 ('line_02', 'item_010'),
 ('line_02', 'item_011'),
 ('line_02', 'item_012'),
 ('line_02', 'item_013'),
 ('line_02', 'item_014'),
 ('line_02', 'item_015'),
 ('line_03', 'item_001'),
 ('line_03', 'item_002'),
 ('line_03', 'item_003'),
 ('line_03', 'item_004'),
 ('line_03', 'item_005'),
 ('line_03', 'item_006'),
 ('line_03', 'item_007'),
 ('line_03', 'item_008'),
 ('line_03',

In [3]:
# PATTERN VISTO COUNT OCCURRENCES check how many times a line appears
logRdd.map(lambda x: (x.get("lineId"),x.get("itemId"))).distinct().map(lambda x: (x[0],1))\
        .reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[0]).collect()

[('line_01', 15),
 ('line_02', 15),
 ('line_03', 15),
 ('line_04', 15),
 ('line_05', 15)]

In [14]:
# most recent
maxDate = logRdd.map(lambda x: x.get("creationDate")).max()
str(maxDate)


'2010-09-18 08:26:07'

In [15]:
# lets add another rdd (masterrdd) with same keys plis a registryinsertts. this simulates a cdc off a master data table.
# it's the ts that signals that the line changed. this is useful when you want to know how data changes over time.
from dataGenerator.rowGenerator import *
from dataGenerator.datasetGenerator import *

dataset = datasetGenerator()
dataset.addGenerator(idGenerator(),prefix="line",min=1,max=5,keyName="lineId")
dataset.addGenerator(idGenerator(),prefix="item",min=1,max=15,keyName="itemId")
dataset.addGenerator(dateGenerator(),endDate=maxDate,keyName="registryInsertTimestamp")
dataset.addGenerator(intGenerator(),"cost")

masterDataRdd = sc.parallelize( dataset.generateDataset(1500)).persist()
masterDataRdd.first()

{'lineId': 'line_03',
 'itemId': 'item_009',
 'registryInsertTimestamp': datetime.datetime(2010, 7, 20, 18, 24, 55),
 'cost': 862}

In [16]:
masterDataRdd.map(lambda x: x.get("registryInsertTimestamp")).max()

datetime.datetime(2010, 9, 18, 2, 31, 5)

# Bean Counter

In [9]:
# this is useful just to know the data structure
logRdd.first()

{'lineId': 'line_05',
 'itemId': 'item_011',
 'creationDate': datetime.datetime(2010, 1, 1, 0, 29, 14),
 'qty': 824}

## How many lines per item?

In [10]:
# how many times an item shows up
logRdd.map(lambda x: (x.get("itemId"),1)).reduceByKey(lambda x,y: x+y).take(5)

[('item_003', 1724),
 ('item_002', 1610),
 ('item_005', 1679),
 ('item_010', 1690),
 ('item_007', 1665)]

## How much qty per item?

In [13]:
# summing the quantity per item
def f(x,y):
    numberOfItems = x+y
    return numberOfItems
logRdd.map(lambda x: (x.get("itemId"),x.get("qty"))).reduceByKey(lambda x,y: f(x,y)).top(100,lambda x: x[0])

[('item_015', 822107),
 ('item_014', 823417),
 ('item_013', 805765),
 ('item_012', 833216),
 ('item_011', 859989),
 ('item_010', 847069),
 ('item_009', 812745),
 ('item_008', 815525),
 ('item_007', 835245),
 ('item_006', 835307),
 ('item_005', 833672),
 ('item_004', 828181),
 ('item_003', 887633),
 ('item_002', 807583),
 ('item_001', 838001)]

In [14]:
# (scritta io) stessa cosa ma senza f
logRdd.map(lambda x: (x.get("itemId"),x.get("qty"))).reduceByKey(lambda x,y: x+y).top(100,lambda x: x[0])

[('item_015', 822107),
 ('item_014', 823417),
 ('item_013', 805765),
 ('item_012', 833216),
 ('item_011', 859989),
 ('item_010', 847069),
 ('item_009', 812745),
 ('item_008', 815525),
 ('item_007', 835245),
 ('item_006', 835307),
 ('item_005', 833672),
 ('item_004', 828181),
 ('item_003', 887633),
 ('item_002', 807583),
 ('item_001', 838001)]

In [15]:
# just to show a more complex final sorti
def f(x,y):
    numberOfItems = x+y
    return numberOfItems
logRdd.map(lambda x: (x.get("itemId"),x.get("qty"))).reduceByKey(lambda x,y: f(x,y))\
                    .top(100,key=lambda x: 1/int(x[0].split("_")[1]))

[('item_001', 838001),
 ('item_002', 807583),
 ('item_003', 887633),
 ('item_004', 828181),
 ('item_005', 833672),
 ('item_006', 835307),
 ('item_007', 835245),
 ('item_008', 815525),
 ('item_009', 812745),
 ('item_010', 847069),
 ('item_011', 859989),
 ('item_012', 833216),
 ('item_013', 805765),
 ('item_014', 823417),
 ('item_015', 822107)]

## How much qty in average per item?

In [16]:
def f(x,y):
    cumulateQty = x[0]+y[0]
    numberOfItems = x[1]+y[1]
    return cumulateQty,numberOfItems
    
def prepare(x):
    return x[0],x[1][0]/x[1][1]
    
logRdd.map(lambda x: (x.get("itemId"),(x.get("qty"),1.0))).reduceByKey(lambda x,y: f(x,y)).map(lambda x: prepare(x)).take(5)

[('item_003', 514.8683294663573),
 ('item_002', 501.60434782608695),
 ('item_005', 496.5288862418106),
 ('item_010', 501.22426035502957),
 ('item_007', 501.64864864864865)]

In [17]:
def f(x,y):
    cumulateQty = x[0]+y[0]
    numberOfItems = x[1]+y[1]
    return cumulateQty,numberOfItems
    
def prepare(x):
    return x[1][0]/x[1][1]
    
logRdd.map(lambda x: (x.get("itemId"),(x.get("qty"),1.0))).reduceByKey(lambda x,y: f(x,y)).map(lambda x: prepare(x))\
                .stats()

(count: 15, mean: 499.4326175141998, stdev: 5.846525274434224, max: 514.8683294663573, min: 490.63290398126463)

## Exercise (1): Similar statistics but on item,line couple

In [25]:
# copying solution presented by a guy (LESSON 13)
# how many lines?
logRdd.map(lambda x: ((x.get("itemId"),x.get("lineId")),1)).reduceByKey(lambda x,y: x+y).take(5)

[(('item_010', 'line_04'), 355),
 (('item_001', 'line_02'), 389),
 (('item_009', 'line_04'), 322),
 (('item_002', 'line_05'), 311),
 (('item_013', 'line_01'), 318)]

In [26]:
# how much quantity?
logRdd.map(lambda x: ((x.get("itemId"),x.get("lineId")),x.get("qty"))).reduceByKey(lambda x,y: x+y).take(5)

[(('item_010', 'line_04'), 168955),
 (('item_001', 'line_02'), 193844),
 (('item_009', 'line_04'), 166712),
 (('item_002', 'line_05'), 157655),
 (('item_013', 'line_01'), 162051)]

In [29]:
# take the prev 2 cells together and divide one by the other
def add_elements(a,b):
    #a = (7, 121238192)
    #b = (12, 9812741) this fun will return (19, ...)
    # a_n would be how many lines so far, and a_qty how much qty so far
    a_n, a_qty = a
    b_n, b_qty = b
    return a_n + b_n, a_qty + b_qty
    
def average(x):
    key, (n, qty) = x
    return qty/n
    
(logRdd
     .map(lambda x: ((x.get("itemId"), x.get("lineId")), (1.0, x.get("qty"))))
     .reduceByKey(add_elements)
     .map(average)
     .stats()
)

(count: 75, mean: 498.4581282803424, stdev: 15.16587378720956, max: 532.9333333333333, min: 460.39506172839504)

## lets Imagine to have 100 executors, and we want to distribute the average over lineId

In [ ]:
# because we have only 15 lineids, spark will split the job on 15 workers, will leaving all other 100 workers idle
def f(x,y):
    cumulateQty = x[0]+y[0]
    numberOfItems = x[1]+y[1]
    return cumulateQty,numberOfItems
    
def prepare(x):
    return x[1][0]/x[1][1]
    
logRdd.map(lambda x: ((x.get("lineId")),(x.get("qty"),1.0))).reduceByKey(lambda x,y: f(x,y))\
                    .map(lambda x: prepare(x))\
                .stats()

In [ ]:
# in order to solve that problem we add a randint to associate each line with a random worker.
# so we split 25000 elements over 100 executors
def f(x,y):
    cumulateQty = x[0]+y[0]
    numberOfItems = x[1]+y[1]
    return cumulateQty,numberOfItems
    
def prepare(x):
    return x[1][0]/x[1][1]
    
logRdd.map(lambda x: ((randint(1,100),x.get("lineId")),(x.get("qty"),1.0))).reduceByKey(lambda x,y: f(x,y))\
                    .map(lambda x: (x[0][1],x[1])).reduceByKey(lambda x,y: f(x,y))\
                    .map(lambda x: prepare(x))\
                .stats()

## Cycle Time = amount of time between the creation of 2 elements SAME AS SLIDES GET CLOSEST ELEMENT

In [4]:
semiLogRdd = logRdd.map(lambda x: ((x.get("lineId"),x.get("itemId")),x)).persist()
semiLogRdd.count()

25000

In [5]:
# join with itself
joinedRDD = semiLogRdd.join(semiLogRdd).persist()
joinedRDD.count()
#8.355.962

8359328

In [6]:
# example of join with itself 
joinedRDD.first()

(('line_04', 'item_002'),
 ({'lineId': 'line_04',
   'itemId': 'item_002',
   'creationDate': datetime.datetime(2010, 1, 1, 0, 32, 21),
   'qty': 923},
  {'lineId': 'line_04',
   'itemId': 'item_002',
   'creationDate': datetime.datetime(2010, 1, 1, 0, 32, 21),
   'qty': 923}))

In [7]:
triangleJoinedRdd = joinedRDD.filter(lambda x: x[1][0].get("creationDate")<x[1][1].get("creationDate")).persist()
triangleJoinedRdd.count()

4167164

In [8]:
# now we have greater date
triangleJoinedRdd.first()

(('line_04', 'item_002'),
 ({'lineId': 'line_04',
   'itemId': 'item_002',
   'creationDate': datetime.datetime(2010, 1, 1, 0, 32, 21),
   'qty': 923},
  {'lineId': 'line_04',
   'itemId': 'item_002',
   'creationDate': datetime.datetime(2010, 1, 1, 5, 16, 17),
   'qty': 455}))

In [9]:
tempRdd = triangleJoinedRdd.map(lambda x: ((x[0],x[1][0].get("creationDate")), 
                                 (x[1][1].get("creationDate")-x[1][0].get("creationDate"),x)))
tempRdd.first()

((('line_04', 'item_002'), datetime.datetime(2010, 1, 1, 0, 32, 21)),
 (datetime.timedelta(seconds=17036),
  (('line_04', 'item_002'),
   ({'lineId': 'line_04',
     'itemId': 'item_002',
     'creationDate': datetime.datetime(2010, 1, 1, 0, 32, 21),
     'qty': 923},
    {'lineId': 'line_04',
     'itemId': 'item_002',
     'creationDate': datetime.datetime(2010, 1, 1, 5, 16, 17),
     'qty': 455}))))

In [10]:
# this shows that amount the 340 possible lines, we chose the one that is closest (first printed below)
sampleLine = tempRdd.first()[0]
print(sampleLine)
triangleJoinedRdd.map(lambda x: ((x[0],x[1][0].get("creationDate")), 
                                 (x[1][1].get("creationDate")-x[1][0].get("creationDate"),x)))\
                .filter(lambda x: x[0]==sampleLine).count()

(('line_04', 'item_002'), datetime.datetime(2010, 1, 1, 0, 32, 21))


340

In [11]:
def f(x,y):
    if x[0]<y[0]:
        return x
    return y

def prepare(x):
    x[1][1][1][0]["cycleTime"] = x[1][0]
    return x[1][1][1][0]
    
enrichedLogRdd = triangleJoinedRdd.map(lambda x: ((x[0],x[1][0].get("creationDate")), 
                                                  (x[1][1].get("creationDate")-x[1][0].get("creationDate"),x)))\
                    .reduceByKey(lambda x,y: f(x,y)).map(lambda x: prepare(x)).persist()
enrichedLogRdd.first()

{'lineId': 'line_04',
 'itemId': 'item_002',
 'creationDate': datetime.datetime(2010, 1, 1, 5, 16, 17),
 'qty': 455,
 'cycleTime': datetime.timedelta(seconds=70096)}

## Exercise (2): why if we count, we see fewer items?

In [ ]:
enrichedLogRdd.count()

In [30]:
"""
Because before when we create triandleJoinedRdd we filtered each pair where
left["creationDate"] < right["creationDate"]
The problem is that for the pair with the highest creationDate there can't be 
any pair where this condition is True. This means that each (itemId, lineId) with 
the highest creationDate is completely removed from the rdd.
This sort of makes sense though, their cycle has not ended yet so we don't add them yet 
to the rdd that stores cycle times 
"""
total_missing_values = logRdd.count() - enrichedLogRdd.count()
print(f"{total_missing_values} values have disappeared from the rdd")

distinct_itemId_lineId = len(logRdd.map(lambda x: ((x.get("itemId"), x.get("lineId")))).distinct().collect())
print(f"There are{distinct_itemId_lineId} distinct (itemId, lineId) pairs")

print("""This makes sense since 1 value is removed for each key""")

75 values have disappeared from the rdd
There are75 distinct (itemId, lineId) pairs
This makes sense since 1 value is removed for each key


## Exercise (3): I want to see the histogram of cycle times

In [ ]:
# GUARDA LESSON 003 IL PROF SPIEGA LI' l'histogram

## Extract Snapshot from Master Data

In [17]:
# snapshot = most recent situation. for each key we must find the most recent value of cost
tempRdd = masterDataRdd.map(lambda x: ((x.get("lineId"),x.get("itemId")), (x.get("cost"),x.get("registryInsertTimestamp")) )  )
tempRdd.first()

(('line_03', 'item_009'), (862, datetime.datetime(2010, 7, 20, 18, 24, 55)))

In [18]:
masterDataRdd.map(lambda x: ((x.get("lineId"),x.get("itemId")))).distinct().count()

75

In [19]:
sampleLine = tempRdd.first()[0]
print(sampleLine)
masterDataRdd.map(lambda x: ((x.get("lineId"),x.get("itemId")), (x.get("cost"),str(x.get("registryInsertTimestamp"))) )  )\
            .filter(lambda x: x[0]==sampleLine).sortBy(lambda x: x[1][1]).collect()

('line_03', 'item_009')


[(('line_03', 'item_009'), (336, '2010-01-26 18:18:27')),
 (('line_03', 'item_009'), (740, '2010-02-09 14:14:32')),
 (('line_03', 'item_009'), (186, '2010-02-10 05:27:24')),
 (('line_03', 'item_009'), (318, '2010-02-20 09:42:32')),
 (('line_03', 'item_009'), (262, '2010-03-01 20:26:54')),
 (('line_03', 'item_009'), (774, '2010-03-18 20:07:38')),
 (('line_03', 'item_009'), (438, '2010-03-23 04:49:49')),
 (('line_03', 'item_009'), (623, '2010-03-26 21:30:15')),
 (('line_03', 'item_009'), (425, '2010-03-31 04:55:31')),
 (('line_03', 'item_009'), (318, '2010-04-21 16:29:29')),
 (('line_03', 'item_009'), (206, '2010-05-31 17:15:42')),
 (('line_03', 'item_009'), (384, '2010-06-06 23:02:44')),
 (('line_03', 'item_009'), (256, '2010-06-12 18:38:53')),
 (('line_03', 'item_009'), (957, '2010-06-18 00:46:14')),
 (('line_03', 'item_009'), (85, '2010-06-23 08:16:27')),
 (('line_03', 'item_009'), (315, '2010-06-30 03:50:29')),
 (('line_03', 'item_009'), (689, '2010-07-14 11:28:41')),
 (('line_03', '

In [20]:
masterDataRdd.count()

1500

In [21]:
masterDataRdd.map(lambda x: ((x.get("lineId"),x.get("itemId")), (x.get("cost"),str(x.get("registryInsertTimestamp"))) )  )\
            .reduceByKey(lambda x,y: x if x[1]>y[1] else y).collect()

[(('line_03', 'item_009'), (862, '2010-07-20 18:24:55')),
 (('line_04', 'item_003'), (154, '2010-09-14 13:09:09')),
 (('line_01', 'item_011'), (945, '2010-09-17 01:38:04')),
 (('line_03', 'item_005'), (890, '2010-09-16 23:22:00')),
 (('line_05', 'item_001'), (797, '2010-09-06 07:12:42')),
 (('line_01', 'item_015'), (879, '2010-08-31 22:46:34')),
 (('line_05', 'item_010'), (544, '2010-09-16 13:02:37')),
 (('line_01', 'item_003'), (993, '2010-09-03 22:29:05')),
 (('line_04', 'item_015'), (209, '2010-08-20 22:53:27')),
 (('line_04', 'item_002'), (643, '2010-09-04 05:39:45')),
 (('line_02', 'item_014'), (958, '2010-09-04 17:10:30')),
 (('line_02', 'item_011'), (585, '2010-09-12 14:17:51')),
 (('line_02', 'item_005'), (719, '2010-09-17 17:24:43')),
 (('line_01', 'item_005'), (770, '2010-09-09 18:46:16')),
 (('line_03', 'item_004'), (78, '2010-09-13 08:23:28')),
 (('line_05', 'item_005'), (398, '2010-08-10 13:17:46')),
 (('line_01', 'item_010'), (133, '2010-09-01 06:10:33')),
 (('line_02', '

# Join over time

In [ ]:
print(sampleLine)
logRdd.map(lambda x: ((x.get("lineId"),x.get("itemId")), str(x.get("creationDate"))) )\
            .filter(lambda x: x[0]==(sampleLine)).sortBy(lambda x: x[1][1]).collect()

## Step1: start with all possible combinations of log/master regardless the timestamp

In [ ]:
step1 = logRdd.map(lambda x: ((x.get("lineId"),x.get("itemId")),x))\
                .join(masterDataRdd.map(lambda x: ((x.get("lineId"),x.get("itemId")),x)))
step1.first()

## Step2: reshape the rdd to create the unique ID needed - (lineId,ItemId,creationDate)

In [ ]:
#log data is leading
def linePrepare(row):
    key = (row[0],row[1][0].get("creationDate"))
    value = {"deltaTime":row[1][0].get("creationDate")-row[1][1].get("registryInsertTimestamp"),
            "logLine":row[1][0],"masterLine":row[1][1]}
    return {"key":key,"value":value}

step2 = step1.map(lambda x: linePrepare(x)).persist()
step2.first()

## Step3: Reducing to Select only the correct value

In [ ]:
def reduceFun(val1,val2):
    return val1 if val1.get("deltaTime") < val2.get("deltaTime") else val2

step3 = step2.map(lambda x: (x.get("key"),x.get("value"))).reduceByKey(lambda x,y: reduceFun(x,y))
step3.count()

In [ ]:
## it looks like correct... lets make a check
step3.filter(lambda x: x[1].get("deltaTime")<timedelta(0)).first() ##a negative value?

In [ ]:
def lineFormatter(row):
    masterLine = row.get("value").get("masterLine")
    return "[{ts}] - {cost}€".format(ts=masterLine.get("registryInsertTimestamp"),cost=masterLine.get("cost"))
sampleRowKey = step2.first().get("key")
print("{line}-{item} @{ts}".format(ts=str(sampleRowKey[1]),
                          line=sampleRowKey[0][0],item=sampleRowKey[0][1]))
print("\t"+"\n\t".join([lineFormatter(k) for k in step2.filter(lambda x: x.get("key") == sampleRowKey).take(20)]))

In [ ]:
sampleRowKey2 = step2.filter(lambda x: x.get("key")[1]>datetime(2010,4,1)).first().get("key")
print("{line}-{item} @{ts}".format(ts=str(sampleRowKey2[1]),
                          line=sampleRowKey2[0][0],item=sampleRowKey2[0][1]))
print("\t"+"\n\t".join([lineFormatter(k) for k in step2.filter(lambda x: x.get("key") == sampleRowKey2)\
                        .top(25,key=lambda x: x.get("value").get("masterLine").get("registryInsertTimestamp"))]))

## Step3: lets try again!

In [ ]:
def reduceFun(val1,val2):
    return val1 if val1.get("deltaTime") < val2.get("deltaTime") else val2
step3_1 = step2.map(lambda x: (x.get("key"),x.get("value"))).filter(lambda x: x[1].get("deltaTime")>=timedelta(0))\
            .reduceByKey(lambda x,y: reduceFun(x,y))
step3_1.count()

why 23883?? we lost some rows

In [ ]:
def lineMaker(row):
    return (row.get("key"),(0,0,1) if row.get("value").get("deltaTime")>timedelta(0) 
                                            else (0,1,0) if row.get("value").get("deltaTime") == 0
                                            else (1,0,0))
    
rowDistribution = step2.map(lambda x: lineMaker(x)).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1],x[2]+y[2])).persist()
[(k[0][0],k[1]) for k in rowDistribution.collect() if k[1][2] == 0]

In [ ]:
len([(k[0][0],k[1]) for k in rowDistribution.collect() if k[1][2] == 0]),len([(k[0][0],k[1]) for k in rowDistribution.collect() if k[1][2] == 0]) + step3_1.count()

## Step3.ter maybe this is the good one

In [ ]:
def reduceFun(val1,val2):
    val1Delta = val1.get("deltaTime")
    val2Delta = val2.get("deltaTime")
    if val1Delta >= timedelta(0) and val2Delta >=timedelta(0):
        return val1 if  val1Delta < val2Delta else val2
    elif val1Delta < timedelta(0) and val2Delta <timedelta(0): 
        return val1 if  val1Delta > val2Delta else val2
    else: 
        return val1 if val1Delta>=timedelta(0) else val2
        
def lineParser(row):
    return {"lineId":row[0][0][0],"itemId":row[0][0][1],"creationDate":row[0][1], "deltaTime":row[1].get("deltaTime"),
                 "qty":row[1].get("logLine").get("qty"),"cost":row[1].get("masterLine").get("cost")}


step3_2 = step2.map(lambda x: (x.get("key"),x.get("value")))\
            .reduceByKey(lambda x,y: reduceFun(x,y)).map(lambda x: lineParser(x))
step3_2.count()

In [ ]:
step3_0 = step3.map(lambda x: lineParser(x))
step3_0.filter(lambda x: x.get("deltaTime")<timedelta(0)).count(),step3_0.filter(lambda x: x.get("deltaTime")>=timedelta(0)).count()

In [ ]:
step3_1_0 = step3_1.map(lambda x: lineParser(x))
step3_1_0.filter(lambda x: x.get("deltaTime")<timedelta(0)).count(),step3_1_0.filter(lambda x: x.get("deltaTime")>=timedelta(0)).count()

In [ ]:
step3_2.filter(lambda x: x.get("deltaTime")<timedelta(0)).count(),step3_2.filter(lambda x: x.get("deltaTime")>=timedelta(0)).count()